In [1]:
import time
import os
import datarobot as dr
import requests

In [2]:
GENAI_API_ROOT = "https://staging.datarobot.com/api-gw/genai"
dr_api_token = os.environ["DATAROBOT_V2_API_TOKEN"]

In [3]:
# Initialize the DataRobot client.
dr.Client(endpoint="https://staging.datarobot.com/api/v2", token=dr_api_token)

In [4]:
# Initialize the GenAI client.
genai_client = requests.session()
genai_client.headers.update({"Authorization": f"Bearer {dr_api_token}"})

In [6]:
# Create a Use Case.
use_case = dr.UseCase.create(name="GenAI Cert 3 Buzok Use Case")

In [7]:
# Create a Playground in the Use Case.
playground_payload = {
    "name": "BYO LLM Playground",
    "description": "This is a playground created using the REST API",
    "useCaseId": use_case.id,
}

response = genai_client.post(f"{GENAI_API_ROOT}/playgrounds/", json=playground_payload)
playground = response.json()
playground

{'id': '653f0c75b4150fd13e2de077',
 'name': 'BYO LLM Playground',
 'description': 'This is a playground created using the REST API',
 'useCaseId': '653f0a57fd42757c7f067e98',
 'creationDate': '2023-10-30T01:52:53.745875+00:00',
 'creationUserId': '5b964766d709b105092df65e',
 'lastUpdateDate': '2023-10-30T01:52:53.745882+00:00',
 'lastUpdateUserId': '5b964766d709b105092df65e',
 'savedLLMBlueprintsCount': 0}

In [8]:
# Create a draft LLM blueprint.
llm_blueprint_payload = {
    "name": "Pre-baked LLM Blueprint",
    "playgroundId": playground["id"],
    "llmId": "azure-openai-gpt-3.5-turbo",
    # Uncomment this if you'd like to use a vector database
    #"vectorDatabaseId": "abcdef0123456789",
}

response = genai_client.post(f"{GENAI_API_ROOT}/llmBlueprints/", json=llm_blueprint_payload)
llm_blueprint = response.json()
llm_blueprint

{'vectorDatabaseId': None,
 'vectorDatabaseSettings': None,
 'llmId': 'azure-openai-gpt-3.5-turbo',
 'llmSettings': {'systemPrompt': '',
  'maxCompletionLength': 1024,
  'temperature': 1.0,
  'topP': 1.0},
 'id': '653f0cd0b4150fd13e2de078',
 'name': 'Pre-baked LLM Blueprint',
 'description': '',
 'isSaved': False,
 'playgroundId': '653f0c75b4150fd13e2de077',
 'llmName': 'Azure OpenAI GPT-3.5 Turbo',
 'creationDate': '2023-10-30T01:54:24.672326+00:00',
 'creationUserId': '5b964766d709b105092df65e',
 'lastUpdateDate': '2023-10-30T01:54:24.672331+00:00',
 'lastUpdateUserId': '5b964766d709b105092df65e',
 'vectorDatabaseName': None,
 'vectorDatabaseStatus': None,
 'vectorDatabaseErrorMessage': None,
 'vectorDatabaseErrorResolution': None,
 'customModelLLMValidationStatus': None,
 'customModelLLMErrorMessage': None,
 'customModelLLMErrorResolution': None}

In [9]:
# Submit a prompt
prompt_payload = {
    "llmBlueprintId": llm_blueprint["id"],
    "text": "What is DataRobot?",
}

response = genai_client.post(f"{GENAI_API_ROOT}/chatPrompts/", json=prompt_payload)

prompt_status_tracking_url = response.headers["Location"]
prompt = response.json()
prompt

{'vectorDatabaseId': None,
 'vectorDatabaseSettings': None,
 'llmId': 'azure-openai-gpt-3.5-turbo',
 'llmSettings': {'systemPrompt': '',
  'maxCompletionLength': 1024,
  'temperature': 1.0,
  'topP': 1.0},
 'id': '653f0cdfdf55afa6b2739178',
 'text': 'What is DataRobot?',
 'llmBlueprintId': '653f0cd0b4150fd13e2de078',
 'creationDate': '2023-10-30T01:54:39.322216+00:00',
 'creationUserId': '5b964766d709b105092df65e',
 'resultMetadata': None,
 'resultText': None,
 'confidenceScores': None,
 'citations': [],
 'executionStatus': 'NEW'}

In [10]:
# Wait for the prompt to complete.
for _ in range(60):
    prompt_status_response = genai_client.get(prompt_status_tracking_url, allow_redirects=False)
    if prompt_status_response.status_code == 303:
        print("Prompt completed successfully")
        break
    else:
        print("Waiting for the prompt to complete...")
        time.sleep(1)

Prompt completed successfully


In [11]:
# Get prompt output.
response = genai_client.get(f"{GENAI_API_ROOT}/chatPrompts/{prompt['id']}/")
completed_prompt = response.json()
print("Prompt output: ", completed_prompt["resultText"])

Prompt output:  DataRobot is an automated machine learning platform that assists organizations in building and deploying accurate predictive models. It utilizes cutting-edge technologies such as artificial intelligence and machine learning algorithms to automate the end-to-end process of designing, training, and deploying models. DataRobot enables users without specialized data science skills to generate actionable insights from complex datasets efficiently. It offers various features, including data preprocessing, automated feature engineering, model selection, hyperparameter tuning, and model deployment.
